# Import Libraries

In [0]:
from __future__ import print_function

from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Input, LSTM, Dense
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import pandas as pd
import os
import io
import time

import string
from string import digits

# Load Data
Download English Arabic dataset. The data is available here: Data source: http://www.manythings.org/anki/

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Load data from repository 
url = '/content/gdrive/My Drive/Colab Notebooks/Machine Translation English to Arabic /ara.txt'
lines = pd.read_csv(url,delimiter="\t",names=['input_texts', 'target_texts', 'notes'])
# Return the DataFrame head
lines.head()

,input_texts,target_texts,notes
0,Hi.,مرحبًا.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
1,Run!,اركض!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
2,Help!,النجدة!,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
3,Jump!,اقفز!,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
4,Stop!,قف!,CC-BY 2.0 (France) Attribution: tatoeba.org #4...


# Explantory Data Analysis

In [0]:
# Return the dimensionality of the DataFrame.
lines.shape

(11320, 3)

In [0]:
# Return information about the DataFrame including the index dtype and column dtypes, non-null values and memory usage.
lines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11320 entries, 0 to 11319
Data columns (total 3 columns):
input_texts     11320 non-null object
target_texts    11320 non-null object
notes           11320 non-null object
dtypes: object(3)
memory usage: 265.4+ KB


# Data Pre-Processing
This section is to perform some data cleaning and preparation.

In [0]:
# Remove a column by specifying label names 
lines = lines.drop(columns='notes')

In [0]:
# Lowercase all characters
lines.input_texts=lines.input_texts.apply(lambda x: x.lower())
lines.target_texts=lines.target_texts.apply(lambda x: x.lower())

In [0]:
# Remove quotes
lines.input_texts=lines.input_texts.apply(lambda x: re.sub("'", '', x))
lines.target_texts=lines.target_texts.apply(lambda x: re.sub("'", '', x))

In [0]:
# Set of all special characters
english_punctuations = set(string.punctuation)
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
# Remove all the special characters
lines.input_texts=lines.input_texts.apply(lambda x: ''.join(ch for ch in x if ch not in english_punctuations))
lines.target_texts=lines.target_texts.apply(lambda x: ''.join(ch for ch in x if ch not in arabic_punctuations))

In [0]:
# Remove Arabic diacritics from text
lines.target_texts=lines.target_texts.apply(lambda x: re.sub("[ ّ َ ً ُ ٌ ِ ٍ ْ]", " ", x))

#  ّ    | # Tashdid
#  َ    | # Fatha
#  ً    | # Tanwin Fath
#  ُ    | # Damma
#  ٌ    | # Tanwin Damm
#  ِ    | # Kasra
#  ٍ    | # Tanwin Kasr
#  ْ    | # Sukun

In [0]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines.input_texts=lines.input_texts.apply(lambda x: x.translate(remove_digits))
lines.target_texts=lines.target_texts.apply(lambda x: re.sub("[١٢٣٤٥٦٧٨٩٠]", "", x))

In [0]:
# Remove extra spaces
lines.input_texts=lines.input_texts.apply(lambda x: x.strip())
lines.target_texts=lines.target_texts.apply(lambda x: x.strip())
lines.input_texts=lines.input_texts.apply(lambda x: re.sub(" +", " ", x))
lines.target_texts=lines.target_texts.apply(lambda x: re.sub(" +", " ", x))

In [0]:
lines.to_csv(r'/content/gdrive/My Drive/Colab Notebooks/Machine Translation English to Arabic /arabic.txt', header=None, index=None, sep='\t', mode='a')

# Text Data Vectorization
This section is to victorize the data and compute the vocabulary for both English and Arabic. Main steps:
- compute the vocabulary sizes,
- compute the length of maximum sequence for both languages,
- create dictionaries to convert a given token into an integer index and vice-versa.

In [0]:
batch_size = 64  # Batch size for training.
epochs = 100 #umber of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 11000  # Number of samples to train on.

In [18]:
# Vectorize the data.
data_path = '/content/gdrive/My Drive/Colab Notebooks/Machine Translation English to Arabic /arabic.txt'
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 11000
Number of unique input tokens: 28
Number of unique output tokens: 79
Max sequence length for inputs: 55
Max sequence length for outputs: 67


In [0]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [0]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]


In [0]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Model Development

In [0]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [23]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.3,shuffle=True)

Epoch 1/100
121/121 [==============================] - 3s 28ms/step - loss: 1.0124 - accuracy: 0.7724 - val_loss: 1.7022 - val_accuracy: 0.6315
Epoch 2/100
121/121 [==============================] - 3s 21ms/step - loss: 0.8075 - accuracy: 0.7863 - val_loss: 1.3115 - val_accuracy: 0.6372
Epoch 3/100
121/121 [==============================] - 3s 21ms/step - loss: 0.7271 - accuracy: 0.8029 - val_loss: 1.2070 - val_accuracy: 0.6767
Epoch 4/100
121/121 [==============================] - 3s 21ms/step - loss: 0.6720 - accuracy: 0.8164 - val_loss: 1.1658 - val_accuracy: 0.6761
Epoch 5/100
121/121 [==============================] - 3s 21ms/step - loss: 0.6484 - accuracy: 0.8214 - val_loss: 1.1435 - val_accuracy: 0.6836
Epoch 6/100
121/121 [==============================] - 3s 21ms/step - loss: 0.6289 - accuracy: 0.8251 - val_loss: 1.0905 - val_accuracy: 0.6993
Epoch 7/100
121/121 [==============================] - 3s 21ms/step - loss: 0.6115 - accuracy: 0.8292 - val_loss: 1.0764 - val_accuracy:

In [0]:
# Save model
model.save('/content/gdrive/My Drive/Colab Notebooks/Machine Translation English to Arabic /s2s_ver6.h5')

In [0]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [0]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [28]:
for seq_index in range(200):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: hi
Decoded sentence: الطاب باليوبور

-
Input sentence: run
Decoded sentence: هل أنت محق

-
Input sentence: help
Decoded sentence: هل حادة متعدما

-
Input sentence: jump
Decoded sentence: استمع

-
Input sentence: stop
Decoded sentence: توقفي عن الصراخ

-
Input sentence: go on
Decoded sentence: استمع

-
Input sentence: go on
Decoded sentence: استمع

-
Input sentence: hello
Decoded sentence: استمع

-
Input sentence: hurry
Decoded sentence: استمع

-
Input sentence: hurry
Decoded sentence: استمع

-
Input sentence: i see
Decoded sentence: استقلت

-
Input sentence: i won
Decoded sentence: استيقظت للتو

-
Input sentence: relax
Decoded sentence: استرح

-
Input sentence: smile
Decoded sentence: انتبه

-
Input sentence: cheers
Decoded sentence: استمع

-
Input sentence: got it
Decoded sentence: هل حدث

-
Input sentence: he ran
Decoded sentence: هو تركني أذهب

-
Input sentence: i know
Decoded sentence: أنا أحدده

-
Input sentence: i know
Decoded sentence: أنا أحدده

-
Input senten